In [1]:
import torch
import matplotlib.pyplot as plt
from torch import nn
import numpy as np
import napari
from deepLearningModels.unet import test
from utils.helper import *
from utils.losses import *
from utils.transformations import stroke_val_transformations, CustomTestStrokTrans
from readConfig import model_configuration
from utils.helper import save_imagen3d, resize_nifti_file
from tqdm import tqdm
import torchvision

c:\Users\vegaarellano\Documents\Gitlab\instancesegmentation\pytorch-img-seg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'deepLearningModels'

In [ ]:
folder = ""


device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = model_configuration["hyper_parameters"]["batch_size"]
image_height = model_configuration["input_image"]["image_height"]
image_width = model_configuration["input_image"]["image_widths"]
num_input_channels = 1
# Load model configuration       

checkpoint = torch.load(model_uri)
learning_rate = checkpoint['learning_rate']
threshold = model_configuration["hyper_parameters"]["threshold"]

#loaded_model = mlflow.pytorch.load_model(model_uri)
# ["UNET", "3DUNET", "5DUNET", "XNET"]
model = getModel(model_configuration["hyper_parameters"]["model"], device	)

# ===== Loss Function ========= 
# ["BCE","Dice Score", "BCE + Dice Score"] 
loss_fn = getLossFunction(model_configuration["hyper_parameters"]["loss_function"])    

# ===== Optimizer =========    
# ["Adam","AdamW", "SGD","RMSprop", "Adagrad"]
type_optimizer = model_configuration["hyper_parameters"]["optimizer"]
optimizer = getOptimizer(type_optimizer,model,model_configuration["optimizer_config"][type_optimizer] )

# ================== Load Model ====================
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epochs']
loss_function = checkpoint['loss_function']